# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [0]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [2]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [5]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./input/genres/{g}'):
        songname = f'./input/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rms = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [6]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00093.wav,0.377690,0.065906,569.930721,995.407125,927.427725,0.021701,-350.436188,169.545746,31.820370,...,1.821690,-5.970891,-5.259567,-0.229211,-1.776850,-3.713751,0.181591,2.072390,-2.896225,blues
1,blues.00087.wav,0.336773,0.158098,1442.190271,1870.534155,3083.414688,0.050889,-155.504929,125.638863,1.596553,...,-0.792893,-7.748057,0.413548,-7.030263,3.997679,-6.256611,0.958227,2.019821,-5.742188,blues
2,blues.00050.wav,0.400860,0.182380,1945.848425,2082.246626,4175.874749,0.085806,-82.979019,107.052124,-25.320452,...,12.539581,-9.762303,2.562253,-6.300853,2.996785,-8.718455,-0.326581,-2.980347,0.712600,blues
3,blues.00044.wav,0.390212,0.136276,2279.124558,2375.102120,5198.360233,0.092570,-109.509285,86.922409,-8.607986,...,11.087481,-5.085794,3.976360,-12.859742,12.343859,0.026216,-0.741568,-5.126620,3.303442,blues
4,blues.00078.wav,0.414188,0.258052,2333.685108,2227.425609,4942.811778,0.123863,-2.524338,101.252716,-33.924385,...,12.506608,-13.368823,6.112817,-9.065890,5.033774,-11.330277,3.166534,-4.567591,-4.033623,blues


In [7]:
data.shape

(1000, 28)

In [8]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [9]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
len(y_train)

800

In [13]:
len(y_test)

200

In [14]:
X_train[10]

array([ 1.25521286,  0.20098883,  0.82091704,  0.00944852,  0.49179319,
        1.8253562 ,  1.16200142, -0.49544522, -1.60464391,  0.70238027,
       -1.7445238 ,  0.44657787, -2.01834113,  1.40397416, -0.73991081,
        2.13537746, -1.52598503,  1.95013416, -1.72251254,  1.37008052,
       -1.2485358 ,  1.29501766, -1.21343263,  2.0484251 , -0.32481925,
        0.97036012])

# Classification with Keras

## Building our Network

In [15]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Epoch 1/20
7/7 [==============================] - 0s 1ms/step - loss: 2.1544 - accuracy: 0.2600
Epoch 2/20
7/7 [==============================] - 0s 2ms/step - loss: 1.8164 - accuracy: 0.3787
Epoch 3/20
7/7 [==============================] - 0s 2ms/step - loss: 1.5851 - accuracy: 0.4212
Epoch 4/20
7/7 [==============================] - 0s 2ms/step - loss: 1.4050 - accuracy: 0.5063
Epoch 5/20
7/7 [==============================] - 0s 2ms/step - loss: 1.2723 - accuracy: 0.5825
Epoch 6/20
7/7 [==============================] - 0s 2ms/step - loss: 1.1651 - accuracy: 0.6025
Epoch 7/20
7/7 [==============================] - 0s 2ms/step - loss: 1.0743 - accuracy: 0.6475
Epoch 8/20
7/7 [==============================] - 0s 1ms/step - loss: 1.0116 - accuracy: 0.6725
Epoch 9/20
7/7 [==============================] - 0s 1ms/step - loss: 0.9498 - accuracy: 0.6925
Epoch 10/20
7/7 [==============================] - 0s 1ms/step - loss: 0.8991 - accuracy: 0.6900
Epoch 11/20
7/7 [======================

In [18]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 764us/step - loss: 1.1047 - accuracy: 0.6500


In [19]:
print('test_acc: ',test_acc)

test_acc:  0.6499999761581421


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [20]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [21]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 0s 54ms/step - loss: 2.3285 - accuracy: 0.1133 - val_loss: 2.1553 - val_accuracy: 0.3200
Epoch 2/30
2/2 [==============================] - 0s 10ms/step - loss: 2.1526 - accuracy: 0.2800 - val_loss: 2.0099 - val_accuracy: 0.3650
Epoch 3/30
2/2 [==============================] - 0s 12ms/step - loss: 2.0195 - accuracy: 0.3083 - val_loss: 1.8719 - val_accuracy: 0.3750
Epoch 4/30
2/2 [==============================] - 0s 9ms/step - loss: 1.8870 - accuracy: 0.3567 - val_loss: 1.7427 - val_accuracy: 0.3950
Epoch 5/30
2/2 [==============================] - 0s 10ms/step - loss: 1.7466 - accuracy: 0.3983 - val_loss: 1.6406 - val_accuracy: 0.4500
Epoch 6/30
2/2 [==============================] - 0s 10ms/step - loss: 1.6046 - accuracy: 0.4800 - val_loss: 1.5668 - val_accuracy: 0.4650
Epoch 7/30
2/2 [==============================] - 0s 10ms/step - loss: 1.4726 - accuracy: 0.5350 - val_loss: 1.4869 - val_accuracy: 0.4800
Epoch 8/30
2/2 [============

In [22]:
results

[1.2831228971481323, 0.5550000071525574]

## Predictions on Test Data

In [23]:
predictions = model.predict(X_test)

In [24]:
predictions[0].shape

(10,)

In [25]:
np.sum(predictions[0])

0.9999999

In [26]:
np.argmax(predictions[0])

2